In [1]:
%matplotlib inline
import csv
import requests
from requests.exceptions import *
import psycopg2
import json
import sqlite3
import time
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
from numpy import array
from lxml import html
from bs4 import BeautifulSoup
from datetime import date,datetime

In [2]:
# Read outfits data

outfits_df = pd.read_csv("outfits.csv")
outfits_df = outfits_df.sort_values('outfit_id').reset_index(drop=True)

outfits_df

,outfit_id,link,title,image,author,create_date,views,likes,description,similar1,similar2,similar3,similar4,similar5
0,2281409,https://www.polyvore.com/cute_colors_white_blu...,Holy Frijoles!,https://ak2.polyvoreimg.com/cgi/img-set/cid/22...,https://kneesock.polyvore.com/,20170914,131,14,A fashion look from July 2008 featuring blue t...,http://www.polyvore.com/knot_even_once/set?id=...,http://www.polyvore.com/gone_grungin/set?id=22...,http://www.polyvore.com/eighth_ave_onward/set?...,http://www.polyvore.com/impact/set?id=228505594,http://www.polyvore.com/easy_fall/set?id=22805...
1,192712867,http://www.polyvore.com/stay_true/set?id=19271...,Stay True!,https://ak2.polyvoreimg.com/cgi/img-set/cid/19...,https://colierollers.polyvore.com/,20160307,618,83,A fashion look from March 2016 featuring strap...,http://www.polyvore.com/captivating_in_cape/se...,http://www.polyvore.com/fall_florals/set?id=22...,http://www.polyvore.com/smell_roses_vintage_fl...,http://www.polyvore.com/vest_for_fall/set?id=2...,http://www.polyvore.com/tweed/set?id=229490466
2,192712937,http://www.polyvore.com/style_9906/set?id=1927...,Style #9906,https://ak2.polyvoreimg.com/cgi/img-set/cid/19...,https://vany-alvarado.polyvore.com/,20160307,2090,471,A fashion look from March 2016 featuring brown...,http://www.polyvore.com/untitled_13732/set?id=...,http://www.polyvore.com/13731/set?id=229561234,http://www.polyvore.com/13730/set?id=229557968,http://www.polyvore.com/13729/set?id=229549027,http://www.polyvore.com/13728/set?id=229548850
3,192712989,http://www.polyvore.com/orange_sets_16/set?id=...,Orange Sets 16,https://ak2.polyvoreimg.com/cgi/img-set/cid/19...,https://malabuenisima.polyvore.com/,20160307,203,84,Sets for Day,http://www.polyvore.com/gucci_floral/set?id=20...,http://www.polyvore.com/basic_pink_versace/set...,http://www.polyvore.com/sumer_blue/set?id=2031...,http://www.polyvore.com/alexander_summer/set?i...,http://www.polyvore.com/balmain_night/set?id=2...
4,192713072,http://www.polyvore.com/bag_style_tweed/set?id...,Bag Style: Tweed,https://ak2.polyvoreimg.com/cgi/img-set/cid/19...,https://stileclassico.polyvore.com/,20160307,6312,264,"A fashion look from March 2016 featuring HUGO,...",http://www.polyvore.com/shimmer_into_fall/set?...,http://www.polyvore.com/hint_autumn/set?id=228...,http://www.polyvore.com/newchic_30/set?id=2284...,http://www.polyvore.com/newchic_26/set?id=2282...,http://www.polyvore.com/newchic_23/set?id=2280...
5,192713239,http://www.polyvore.com/05.03/set?id=192713239,ZAFUL #3 05.03,https://ak1.polyvoreimg.com/cgi/img-set/cid/19...,https://edita-m.polyvore.com/,20160307,560,204,@zaful\n#zaful\n\nDress: http://www.zaful.com/...,http://www.polyvore.com/shein/set?id=225996864,http://www.polyvore.com/yellow/set?id=220065363,http://www.polyvore.com/white_dress/set?id=220...,http://www.polyvore.com/romwe/set?id=220000419,http://www.polyvore.com/shein/set?id=219735126
6,192713613,http://www.polyvore.com/ya_rayah_rachid_taha/s...,"Ya Rayah, Rachid Taha",https://ak2.polyvoreimg.com/cgi/img-set/cid/19...,https://blendasantos.polyvore.com/,20160307,1330,194,#stripes,http://www.polyvore.com/doce_ivete_sangalo/set...,http://www.polyvore.com/waiting_feat_noah_cyru...,http://www.polyvore.com/vuelve_feat_daddy_yank...,http://www.polyvore.com/di%C3%A1spora_tribalis...,http://www.polyvore.com/alphabeat_david_guetta...
7,192713940,http://www.polyvore.com/style_9907/set?id=1927...,Style #9907,https://ak1.polyvoreimg.com/cgi/img-set/cid/19...,https://vany-alvarado.polyvore.com/,20160307,2295,434,A fashion look from March 2016 featuring pleat...,http://www.polyvore.com/untitled_13736/set?id=...,http://www.polyvore.com/untitled_13735/set?id=...,http://www.polyvore.com/untitled_13734/set?id=...,http://www.polyvore.com/untitled_13733/set?id=...,http://www.polyvore.com/untitled_13732/set?id=...
8,192714043,http://www.polyvore.com/untitled_3872/set?id=1...,Untitled #3872,https://ak1.polyvoreimg.com/cgi/img-set/cid/19...,https://keliseblog.polyvore.com/,2016

In [3]:
len(outfits_df)

126079

In [6]:
# day = datetime.now().strftime("%Y%m%d")
day = 20180122
column_name1 = 'views_' + str(day)
column_name2 = 'likes_' + str(day)

print column_name1, column_name2

views_20180122 likes_20180122


In [7]:
# Connect to postgres database
conn = psycopg2.connect(database="fashion", user="michellechen", password="", host="127.0.0.1", port="5432")

print "Opened database successfully"

# Connection cursor to database
cur = conn.cursor()

# cur.execute('''ALTER TABLE outfits ADD ''' + column_name1 + ''' INT;''')
# cur.execute('''ALTER TABLE outfits ADD ''' + column_name2 + ''' INT;''')

# conn.commit() 

Opened database successfully


In [8]:
# Headers

user_agent_list = [\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"\
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",\
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",\
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",\
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",\
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",\
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.65 Safari/537.36",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",\
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",\
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
       ]


In [ ]:
i = 16375
while i <= 42000:
    
    outfit_id = outfits_df.loc[i][0]
    outfit_link = outfits_df.loc[i][1]
    outfit_views = 0
    outfit_likes = 0
    
    print i, outfit_id
    
    time.sleep(random.uniform(0, 1))
    head = {'User-Agent': random.choice(user_agent_list)}
    
    # Get request response
    while True:
        try:
            res = requests.get(outfit_link, headers = head)
        except RequestException:
            print "Outfit Request Connection Error!"
            print "Delay 1 minutes!"
            # Delay 60 seconds
            time.sleep(60) 
        else:
            break

    # If outfit page is not found
    if res.status_code == 404:
        # Skip to next outfit
        print "HTTP Response Error " + str(res.status_code)
        print "Outfit Not Found!"
        
    else:        
        # When http response error
        while res.status_code != 200:
            # Print error code
            print "HTTP Response Error " + str(res.status_code)
            # Delay 5 seconds
            time.sleep(5) 
            # Requests again
            res = requests.get(outfit_link, headers = head) 
            if res.status_code == 200:
                break

        # Parse html with BeautifulSoup
        soup = BeautifulSoup(res.text, "html.parser")
        
        # Outfit views count
        outfit_views = int(str(soup.select('div.meta')[0].select('div')[-1])[5:-7].split('.')[1].split()[0].replace(',', ''))
        
        # Outfit likes count
        outfit_likes = int(soup.find_all('meta', {'property': 'polyvore:likes'})[0]['content'])
        
        # Update likes count in database
        cur.execute( '''UPDATE outfits SET ''' + 
                        column_name1 + ''' = ''' + str(outfit_views) + ''', ''' + 
                        column_name2 + ''' = ''' + str(outfit_likes) + ''' 
                        WHERE outfit_id = ''' + str(outfit_id) + ''';''')
                
        conn.commit()
    
    i = i + 1

16375 199360250
16376 199360251
16377 199360254
16378 199360259
16379 199360260
16380 199360273
16381 199360277
16382 199360287
16383 199360289
16384 199360314
16385 199360329
16386 199360361
16387 199360382
16388 199360383
16389 199360400
16390 199360409
16391 199360414
16392 199360417
16393 199360419
16394 199360425
16395 199360436
16396 199360468
16397 199360475
16398 199360476
16399 199360486
16400 199360507
16401 199360510
16402 199360518
16403 199360554
16404 199360560
16405 199360570
16406 199360580
HTTP Response Error 404
Outfit Not Found!
16407 199360607
16408 199360611
16409 199360623
16410 199360629
16411 199360638
16412 199360650
16413 199360651
16414 199360664
16415 199360674
16416 199360699
16417 199360703
16418 199360710
16419 199360761
16420 199360794
16421 199360811
16422 199360813
16423 199360814
16424 199360828
16425 199360830
16426 199360844
16427 199360862
16428 199360868
16429 199360893
16430 199360895
16431 199360909
16432 199360913
16433 199360914
16434 19936092